<a href="https://colab.research.google.com/github/Luisinho-31/reddit_Violencia-Obst-trica/blob/main/Reddit_VO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reddit Crawler

El siguiente código extrae publicaciones de Reddit realizando una búsqueda utilizando un filtrado de palabras que coinciden con el tema de Violencia Obstétrica

In [8]:
# Instalamos librerias necesarias
!pip install praw
!pip install langdetect

In [9]:
# Importamos las librerías, la librería praw es la que nos dará el acceso a Reditt a través de su API
import pandas as pd
import praw
from langdetect import detect
import json
import numpy
import os
import re

In [10]:
#Para tener acceso a la API de Redit utilizaremos las siguiente credenciales, que se obtienen al registrarse como desarrollador
reddit = praw.Reddit(
    client_id='OBHT6tqFHDTwynzafp9Plw',
    client_secret='awYnSJd8SEQxfGCVbfhuG6pzbvAq6g',
    user_agent='HughWarrior'
)

In [11]:
lista_de_texto = ['Violencia Obstetrica', 'Cesáreas innecesarias', 'CesáreasInnecesarias',
 'parir', 'Trabajo de parto', 'TrabajoParto', 'Cuello del utero', 'CuellodelUtero',
 'Centímetros de Dilatación', 'CentímetrosDilatación', 'Desgarre Cervix', 'DesgarreCervix',
 'Episiotomía', 'Maniobra de Kristeller', 'ManiobraKristeller', 'Complicación Embarazo',
 'ComplicaciónEmbarazo', 'Complicación Parto', 'ComplicaciónParto', 'Complicación Cesárea',
 'Complicación Cesárea', 'Discriminación por Edad',
 'Discriminación por Edad',
 'Discriminación por Sobrepeso',
 'DiscriminaciónSobrepeso',
 'Esterilización Forzada',
 'EsterilizaciónForzada',
 'Sufrimiento Fetal',
 'SufrimientoFetal',
 'Parto Inducido',
 'PartoInducido',
 'Tactos Vaginales',
 'TactosVaginales',
 'Falta de Intimidad',
 'FaltaIntimidad',
 'Acelerar el parto',
 'Acelerarelparto',
 'Oxitocina Sintetica',
 'OxitocinaSintetica',
 'Comentarios Irónicos',
 'ComentariosIrónicos',
 'Trato Deshumanizado',
 'TratoDeshumanizado',
 'Anestesia Epidural',
 'AnestesiaEpidural',
 'Ruptura de Membranas',
 'Ruptura de Membranas',
 'Anestecia General innecesaria',
 'AnesteciaGeneralInnecesaria',
 'Castigo por Quejarme',
 'CastigoporQuejarme',
 'Castigo por Gritar',
 'CastigoporGritar',
 'Regaños del médico',
 'RegañosdelMédico',
 'Sentirme Ignorada por el Médico',
 'SentirmeIgnoradaporelMédico',
 'Separación del Bebé',
 'Separación del Bebé',
 'Apego Inmediato del bebé',
 'ApegoInmediatodelBebé',
 'Hemorragia Parto',
 'HemorragiaParto',
 'Falta de consentimiento informado',
 'FaltadeConsentimientoInformado',
 'Ignorar mi dolor',
 'IgnorarDolor',
 'Ejercicios de Médicos Residentes',
 'EjerciciosMédicosResidentes',
 'Ejercicios de estudiantes de medicina',
 'Ejerciciosdeestudiantesdemedicina',
 'Intervención médica innecesaria',
 'Intervenciónmédicainnecesaria',
 'No permitir el apego inmediato madre bebé',
 'Nopermitirelapegoinmediatomadrebebé',
 'Trato deshumanizado',
 'TratoDeshumanizado',
 'Parto humanizado',
 'PartoHumanizado',
 'Parto respetado',
 'PartoRespetado',
 'parteras',
 'doulas',
 'Trato grosero médico',
 'Tratogroseromédico',
 'Sufrimiento innecesario',
 'Sufrimiento innecesario']

In [14]:
# Definimos la lista de palabras a utilizar para realizar la búsqueda
# f = open('PalabrasVO.txt','r', encoding ='utf-8' )
# palabras = f.read()#.replace("#", "")
# list_word = palabras.split("\n")
# list_word
list_word = lista_de_texto

In [15]:
#La siguiente función hace que al realizar la búsqueda se omitan todos aquellos caracteres o emojis que existan dentro de una publicación
def filtrar_texto(texto):
    return re.sub(r'[^\w\s]', '', texto)

#Función para hacer un filtrado del listado de palabras utilizando expresiones regulares, y obtener una mejor búsqueda
def filterNoteByPattern(note):
    #Expresiones Regulares de cada una de las palabras antes cargadas
    p = r"\bviolen[cs]ia\b"
    p += r"|\bces[á|a]reas inn?ecesarias\b"
    p += r"|\btrabajodeparto\b"
    p += r"|\b(Trabajo.*\bParto\b )\b"
    p += r"|\bCuellodelutero\b"
    p += r"|\bDesgarre.*Cervix\b"
    p += r"|\bcent[í|i]metros.*\s?dilataci[o|ó]n\b"
    p += r"|\bFalta\s?.*Intimidad\b"
    p += r"|\bIgnorar\s?.*Dolor\b"
    p += r"|\bTrato\s?.*deshumanizado\b"
    p += r"|\bRuptura\s?de\s?Membranas\b"
    p += r"|\bCuellodelutero\b"
    p += r"|\bHemorragia\s?.*Parto\b"
    p += r"|\bEjercicios\s?M[é|e]dicos\s?Residentes\b"
    p += r"|\bparteras\b"
    p += r"|\bdoulas\b"
    p += r"|\bTrato\s?grosero\s?m[é|e]dico\b"
    p += r"|\bces[á|a]reas inn?ecesarias\b"
    p += r"|\btrabajodeparto\b"
    p += r"|\b(Trabajo.*\bParto\b )\b"
    p += r"|\b(Cuello .*\bUtero\b)\b"
    p += r"|\bCuellodelutero\b"
    p += r"|\bcent[í|i]metros.*\s?dilataci[o|ó]n\b"
    p += r"|\bDesgarre.*Cervix\b"
    p += r"|\bEpisiotom[í|i]a\b"
    p += r"|\bManiobra.*\s?Kristeller\b"
    p += r"|\bcomplicaci[ó|o]n\s?.*emba?razo\b"
    p += r"|\bComplicaci[ó|o]n\s?Parto\b"
    p += r"|\bComplicaci[ó|o]n.*Ces[á|a]rea\b"
    p += r"|\bDiscriminaci[ó|o]n\s?.*Edad\b"
    p += r"|\bDiscriminaci[ó|o]n.*sobrepeso\b"
    p += r"|\bEsterilizaci[ó|o]n Forzada\b"
    p += r"|\b(sufrimiento\s?fetal)\b"
    p += r"|\bParto\s?Inducido\b"
    p += r"|\bTactos\s?Vaginales\b"
    p += r"|\bFalta\s?.*Intimidad\b"
    p += r"|\bAcelerar\s?el\s?parto\b"
    p += r"|\bOxitocina\s?Sintetica\b"
    p += r"|\bComentarios\s?Ir[ó|o]nicos\b"
    p += r"|\bTrato\s?.*deshumanizado\b"
    p += r"|\bAnestesia\s?Epidural\b"
    p += r"|\bRuptura\s?de\s?Membranas\b"
    p += r"|\bAneste[sc]ia\s?General\s?Inn?ecesaria\b"
    p += r"|\bCastigo\s?por\s?Quejarme\b"
    p += r"|\bCastigo\s?por\s?Gritar\b"
    p += r"|\bRegaños\s?del\s?M[é|e]dico\b"
    p += r"|\bSentirme\s?Ignorada\s?por\s?el\s?M[é|e]dico\b"
    p += r"|\bSeparaci[ó|o]n\s?del\s?Beb[é|e]\b"
    p += r"|\bApego\s?Inmediato\s?del\s?Beb[é|e]\b"
    p += r"|\bHemorragia\s?.*Parto\b"
    p += r"|\bFalta\s?de\s?Consentimiento\s?Informado\b"
    p += r"|\bIgnorar\s?.*Dolor\b"
    p += r"|\bEjercicios\s?M[é|e]dicos\s?Residentes\b"
    p += r"|\bEjercicios\s?de\s?estudiantes\s?de\s?medicina\b"
    p += r"|\bIntervenci[ó|o]n\s?m[é|e]dica\s?inn?ecesaria\b"
    p += r"|\bNo\s?permitir\s?el\s?apego\s?inmediato\s?.*madre\s?.*bebé\b"
    p += r"|\bParto\s?Humanizado\b"
    p += r"|\bParto\s?Respetado\b"
    p += r"|\bparteras\b"
    p += r"|\bdoulas\b"
    p += r"|\bTrato\s?grosero\s?m[é|e]dico\b"
    p += r"|\bSufrimiento\s?inn?ecesario\b"
    p += r"|\balarma\s?obst[é|e]trica\b"

    patterns = r"" + p
    matches = re.finditer(pattern=patterns, string=note, flags=re.I)
    #Definimos la variable para guardar cada coincidencia encontrada
    result = set()
    for matched in matches:
        if matched.group() not in result:
            result.add(matched.group())
    return result

#variable donde almacenaremos las publicaciones que coinciden con alguna de las palbras definidas en nuestra lista de palabras después de pasar por la función de patrones definida como filterNoteByPattern
comentario = []

#Se realiza la búsqueda en reddi
for palabra in list_word:
    try:
        #Parámetro para definir los subreddits que se van a explorar, parámetro 'all' significa que busque en todos los subreddits de la red
        subreddit = reddit.subreddit('all')
        #función .search para realizar la búsqueda, parámetro 'limit' se utiliza como filtro para cantidad de publicaciones a explorar, se puede incrementar o disminuir
        posts = subreddit.search(palabra, limit = 200)

        for publicacion in posts:
            titulo = filtrar_texto(publicacion.title)
            selftext = filtrar_texto(publicacion.selftext)

            #FIltramos por publicaciones que se encuentren únicamente en español
            if detect(titulo) or detect (selftext) == 'es':
                # Aplica las expresiones regulares de filterNoteByPattern a título y contenido
                pattern_result_title = filterNoteByPattern(titulo)
                pattern_result_selftext = filterNoteByPattern(selftext)

            # Verifica si hay coincidencias en el título o el contenido con la función que define las expresiones regulares
            if pattern_result_title or pattern_result_selftext:
                comentarios_iniciales = {"titulo": titulo, 'contenido': selftext, 'URL': publicacion.url,
                                          'patron_resultado_titulo': pattern_result_title,
                                          'patron_resultado_contenido': pattern_result_selftext}
                comentario.append(comentarios_iniciales)
    #imprime aquelas palabras que no coinciden o no se encontraron dentro de alguna publicación
    except Exception as error:
        print(f"Error al procesar la palabra {palabra}: {str(error)}")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Error al procesar la palabra parir: No features in text.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Error al procesar la palabra Apego Inmediato del bebé: No features in text.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Error al procesar la palabra Ignorar mi dolor: No features in text.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Error al procesar la palabra IgnorarDolor: No features in text.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [16]:
#Guardamos en un dataframe nuestros resultados
df = pd.DataFrame(comentario)
#Imprime el número de coincidencias
print(df.shape)

(1200, 5)


In [17]:
#Visualizamos el dataframe
df

,titulo,contenido,URL,patron_resultado_titulo,patron_resultado_contenido
0,Soy periodista y estoy investigando sobre la v...,\nEstoy realizando una encuesta para una inves...,https://www.reddit.com/r/RedditPregunta/commen...,{violencia},{violencia}
1,Descubri que estoy embarazada y decidi abortar...,Les comento mi situacion\nTengo menos de 25 y ...,https://www.reddit.com/r/AbortoLegal/comments/...,{violencia},{}
2,Está wea es una vergüenza,gobierno feminista decían cuando al final del ...,https://i.redd.it/c6f5gxg8xd2f1.jpeg,{},{violencia}
3,El embarazo es realmente tan horrible,Tengo 16 actualmente estoy en una relación ser...,https://www.reddit.com/r/AskArgentina/comments...,{},{violencia}
4,Ansiedad por el parto,Hola Este post es en especial para madres y pa...,https://www.reddit.com/r/Ticos/comments/1998hd...,{},{violencia}
...,...,...,...,...,...
1195,Cómo volverte un manifestador poderoso,\n\nUn manifestador poderoso es una persona ...,https://www.reddit.com/r/DesarrolloPersonal/co...,{},{sufrimiento innecesario}
1196,Días tristes,El bajón ha llegado sabía que llegaría tenía q...,https://www.reddit.com/r/espanol/comments/hn9i...,{},{sufrimiento innecesario}
1197,Círculo de SANIDAD de Madrid RESOLUCIÓN Por un...,Nos han contado que es importante garantizar l...,https://www.reddit.com/r/podemos/comments/2ist...,{},{violencia}
1198,La plenitud y sus características más importantes,La plenitud es uno de los propósitos más impor...,https://www.reddit.com/r/DesarrolloPersonal/co...,{},{sufrimiento innecesario}


In [ ]:
#Convertimos nuestro dataframe a un archivo a excel
# df.to_excel('Reddit_Dataset.xlsx', index=False)